<h1> Retrieve data from qdrant</h1>


In [1]:
from src.gen_pipeline import GenPipeline
from src.utils import make_filter

gen_pipeline = GenPipeline()
index = gen_pipeline._get_qdrant_index()

/home/osama/Desktop/intermediate-pilot/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/osama/Desktop/intermediate-pilot/venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
query = "What is the fund size of Alfalah GHP Equity Sub Fund of Alfalah GHP Pension Fund in may 2020"
retriever = index.as_retriever(similarity_top_k=50, filters=make_filter(query))
retrieve_nodes = retriever.retrieve(query)
retrieve_nodes

[NodeWithScore(node=TextNode(id_='93fd5d01-51bf-47d0-9bb6-95f68388f296', embedding=None, metadata={'year': '2020', 'month': 'May', 'filename': 'Islamic – Fund Manager Report – May 2020_14.md.md', 'file_id': 'a475af1c-a7a6-4d77-ac9d-cfa529f73fa9'}, excluded_embed_metadata_keys=['file_id'], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a475af1c-a7a6-4d77-ac9d-cfa529f73fa9', node_type=None, metadata={'filename': 'Islamic – Fund Manager Report – May 2020_14.md.md'}, hash=None)}, text='Fund Statistics of Alfalah GHP Islamic Dedicated Equity Fund\n| Fund Size (PkR mn; as on 31 May, 2020):           | 0.0    |\n| ------------------------------------------------- | ------ |\n| Fund Size including fund of funds (Pkr mn)\\*\\*\\*: | 344.33 |\n| NAV (PkR):                                        | 64.23  |\n| Sharpe Ratio:                                     | 0.90   |\n| Information Ratio:                                | -1.65  |\n| Standa

<h1>Format retrieved chunks</h1>


In [ ]:
def format_retrieved_chunks(retrieved_chunks):
    formatted_texts = []

    for node_with_score in retrieved_chunks:
        node = node_with_score.node
        metadata = node.metadata

        # Extract metadata
        year = metadata.get("year", None)
        month = metadata.get("month", None)
        filename = metadata.get("filename", "N/A")
        text_metadata = metadata.get("text_metadata", None)
        text_content = node.text

        # Format text according to the desired output
        formatted_text = ""

        if year:
            formatted_text += f"year: {year}\n"
        if month:
            formatted_text += f"month: {month}\n"

        formatted_text += f"filename: {filename}\n"

        if text_metadata:
            formatted_text += f"text_content: {text_content}\n{text_metadata}\n------------------------------"
        else:
            formatted_text += (
                f"text_content: {text_content}\n--------------------------"
            )
        formatted_texts.append(formatted_text)

    # Join all the formatted texts together
    return "\n".join(formatted_texts)


# Format the retrieved chunks and print them
formated_nodes = format_retrieved_chunks(retrieve_nodes)

<h1>Llama index Token Counter</h1>


In [1]:
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
from llama_index.core import Settings
import tiktoken

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

Settings.callback_manager = CallbackManager([token_counter])
print("prompt: ", token_counter.llm_token_counts[0].prompt[:100], "...\n")
print(
    "prompt token count: ",
    token_counter.llm_token_counts[0].prompt_token_count,
    "\n",
)

print("completion: ", token_counter.llm_token_counts[0].completion[:100], "...\n")
print(
    "completion token count: ",
    token_counter.llm_token_counts[0].completion_token_count,
    "\n",
)

print("total token count", token_counter.llm_token_counts[0].total_token_count)

IndexError: list index out of range

In [ ]:
print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
    token_counter.llm_token_counts,>
)

<h1>Cost Calculator</h1>


In [2]:
# Define the costs per token in dollars
prompt_token_cost = 0.50 / 1_000_000  # $0.50 per 1M tokens
completion_token_cost = 1.50 / 1_000_000  # $1.50 per 1M tokens

# Define the number of tokens
prompt_tokens = 5866
completion_tokens = 48

# Calculate the cost
total_prompt_cost = prompt_tokens * prompt_token_cost
total_completion_cost = completion_tokens * completion_token_cost
total_cost = total_prompt_cost + total_completion_cost

# Print the results
print(f"Total cost: ${total_cost:.6f}")

Total cost: $0.003005


<h1>Reset token counter</h1>


In [14]:
token_counter.reset_counts()

<h1>LLM Rerank</h1>


In [13]:
from llama_index.core.postprocessor import LLMRerank
from pprint import pprint
from llama_index.llms.openai import OpenAI

llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

# llm = ChatOpenAI(
#     model=self.model_name,
#     temperature=0.0,
#     verbose=True,
#     streaming=True,
#     stream_usage=True,
# )

postprocessor = LLMRerank(choice_batch_size=10, top_n=5)
pprint(postprocessor)
# postprocessor = LLMRerank(choice_batch_size=10, top_n=5)
rerank_retrieve_nodes = postprocessor.postprocess_nodes(retrieve_nodes, query_str=query)
print(type(rerank_retrieve_nodes))
rerank_retrieve_nodes

LLMRerank(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ed70b1e6110>, top_n=5, choice_select_prompt=PromptTemplate(metadata={'prompt_type': <PromptType.CHOICE_SELECT: 'choice_select'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template="A list of documents is shown below. Each document has a number next to it along with a summary of the document. A question is also provided. \nRespond with the numbers of the documents you should consult to answer the question, in order of relevance, as well \nas the relevance score. The relevance score is a number from 1-10 based on how relevant you think the document is to the question.\nDo not include any documents that are not relevant to the question. \nExample format: \nDocument 1:\n<summary of document 1>\n\nDocument 2:\n<summary of document 2>\n\n...\n\nDocument 10:\n<summary of document 10>\n\nQuestion: <question>\nAnswer:\nDoc

[]

<h1> LLMRank error handling functions</h1>


In [18]:
import logging as logger
from llama_index.core.postprocessor import LLMRerank


def rerank_retrieve_nodes(retrieve_nodes, query_str, reranker):
    attempts = 0
    retries = 3
    while attempts < retries:
        try:
            # Perform reranking
            reranked_nodes = reranker.postprocess_nodes(
                retrieve_nodes, query_str=query_str
            )
            print(f"reranked_nodes = {reranked_nodes}")
            logger.info(f"reranked_nodes = {reranked_nodes}")
            return reranked_nodes  # Exit loop on success
        except (ValueError, IndexError) as e:
            # Log specific error details
            attempts += 1
            logger.error(
                f"Error during reranking on attempt {attempts}/{retries}: {type(e).__name__} - {e}"
            )
        except Exception as e:
            # Catch any other unexpected errors
            attempts += 1
            logger.error(
                f"Unexpected error during reranking on attempt {attempts}/{retries}: {type(e).__name__} - {e}"
            )
    logger.error(f"Reranking failed after all retries. on query: '{query_str}'")
    return []

In [19]:
reranker = LLMRerank(choice_batch_size=10, top_n=5)
rerank_retrieve_nodes(retrieve_nodes, query, reranker)

ERROR:root:Error during reranking on attempt 1/3: IndexError - list index out of range
ERROR:root:Error during reranking on attempt 2/3: IndexError - list index out of range
ERROR:root:Error during reranking on attempt 3/3: IndexError - list index out of range
ERROR:root:Reranking failed after all retries. on query: 'great'


[]

In [ ]:
import time


def rerank_retrieve_nodes(retrieve_nodes, query_str, llm):
    attempts = 0
    retries = 3
    while attempts < retries:
        try:
            # Initialize LLMRerank postprocessor
            postprocessor = LLMRerank(choice_batch_size=10, top_n=5, llm=llm)

            # Perform reranking
            reranked_nodes = postprocessor.postprocess_nodes(
                retrieve_nodes, query_str=query_str
            )
            print("--------------------------------")
            # print(reranked_nodes)
            return reranked_nodes  # Exit loop on success
        except (ValueError, IndexError) as e:
            # Log specific error details
            attempts += 1
            logging.error(
                f"Error during reranking on attempt {attempts}/{retries}: {type(e).__name__} - {e}"
            )
        except Exception as e:
            # Catch any other unexpected errors
            attempts += 1
            logging.error(
                f"Unexpected error during reranking on attempt {attempts}/{retries}: {type(e).__name__} - {e}"
            )
    logging.error("Reranking failed after all retries. Returning None.")
    return None

In [86]:
rerank_retrieve_nodes(retrieve_nodes, query, llm, retries=3, delay=2)

--------------------------------


[]

In [21]:
from llama_index.core.postprocessor import SentenceTransformerRerank

# We choose a model with relatively high speed and decent accuracy.
postprocessor = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=5
)

retrieve_nodes = postprocessor.postprocess_nodes(retrieve_nodes, query_str=query)

/home/osama/Desktop/intermediate-pilot/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


<h1> SimilarityPostprocessor Reranker </h1>


In [ ]:
from llama_index.core.postprocessor import SimilarityPostprocessor

postprocessor = SimilarityPostprocessor(similarity_cutoff=0.7)

postprocessor.postprocess_nodes(retrieve_nodes, query_str=query)

<h1> Long context reorder</h1>


In [ ]:
from llama_index.core.postprocessor import LongContextReorder

postprocessor = LongContextReorder()

postprocessor.postprocess_nodes(retrieve_nodes, query_str=query)

<h1> Create nodes data and nodes from input dir(handle best performing funds too)</h1>


In [5]:
import re
import os
import pandas as pd

from langchain_text_splitters import MarkdownHeaderTextSplitter
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file.flat import FlatReader
from llama_index.core.schema import TextNode, RelatedNodeInfo, NodeRelationship
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition,
)
from deep_translator import GoogleTranslator
from src.config import MONTH_FULL_NAMES, MONTH_PATTERN, YEAR_PATTERN
from src.utils import extract_month, extract_year


def creat_node_data_from_input_dir(inpur_dir):

    documents = SimpleDirectoryReader(
        input_dir=inpur_dir,
        file_extractor={
            ".md": FlatReader()
        },  # This disables the MarkdownReader for .md files
        recursive=True,
    ).load_data()

    nodes_data = []

    for document in documents:
        markdown_document = document.get_content()
        filename = document.metadata.get("filename")
        file_id = document.id_
        headers_to_split_on = [
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ]
        markdown_splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=headers_to_split_on
        )
        md_header_splits = markdown_splitter.split_text(markdown_document)

        node_data = {"file_id": file_id, "filename": filename, "node_text": []}

        for text in md_header_splits:
            headers_combined = []

            # Loop through metadata and concatenate headers
            for _, header in text.metadata.items():
                if header:
                    headers_combined.append(header)

            headers_combined = " of ".join(headers_combined[::-1])
            # Concatenate headers and page content
            concat_text = headers_combined + "\n" + text.page_content
            node_data["node_text"].append(concat_text)
        nodes_data.append(node_data)

    return nodes_data


# pattern = r"(Top Performing Funds and Returns for .+)\n(title\|subtitle\|percent\|timeperiod\n([\w\s\-\–().|%–:]+))"
# pattern = r"(# .+?)\n+(\| title\s+\| subtitle\s+\| percent\s+\| timeperiod\s+\|[\s\S]+?)(?=\n\s*#|\Z)"
pattern = r"^(Top Performing Funds and Returns for .+?)\n(\|.+?)(?=\n[A-Z#]|$)"


def create_nodes_from_nodes_data(nodes_data):
    nodes = []
    for data in nodes_data:
        filename = data["filename"]
        month = extract_month(filename)
        year = extract_year(filename)
        if month and year:
            node_text = data["node_text"][0]
            match = re.search(pattern, node_text, re.DOTALL)
            if match and len(data["node_text"]) == 1:
                print(f'lenght of nodes {len(data["node_text"])}')
                report_title = match.group(1)
                report_content = match.group(2)
                node = TextNode(
                    text=report_title,
                    metadata={
                        "year": str(year),
                        "month": month,
                        "filename": filename,
                        "file_id": data["file_id"],
                        "text_metadata": report_content,
                    },
                )
                node.excluded_embed_metadata_keys = ["text_metadata", "file_id"]
                print("iam here")
                node.relationships[NodeRelationship.SOURCE] = RelatedNodeInfo(
                    node_id=data["file_id"], metadata={"filename": filename}
                )
                nodes.append(node)
            else:
                for text in data["node_text"]:
                    node = TextNode(
                        text=(text),
                        metadata={
                            "year": str(year),
                            "month": month,
                            "filename": filename,
                            "file_id": data["file_id"],
                        },
                    )
                    node.excluded_embed_metadata_keys = ["file_id"]
                    node.relationships[NodeRelationship.SOURCE] = RelatedNodeInfo(
                        node_id=data["file_id"], metadata={"filename": filename}
                    )
                    nodes.append(node)
        else:
            for text in data["node_text"]:
                node = TextNode(
                    text=(text),
                    metadata={
                        "filename": filename,
                        "file_id": data["file_id"],
                    },
                )
                node.excluded_embed_metadata_keys = ["file_id"]
                node.relationships[NodeRelationship.SOURCE] = RelatedNodeInfo(
                    node_id=data["file_id"], metadata={"filename": filename}
                )

                nodes.append(node)

    return nodes

<h1> Delete data from Qdrant</h1>


In [1]:
import logging
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# from src.utils import creat_node_data_from_input_dir, create_nodes_from_nodes_data
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, models
import qdrant_client
from llama_index.embeddings.openai import OpenAIEmbedding

/home/user/Desktop/intermediate-pilot/intermediate-pilot/intermediate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def delete_data_from_qdrant(filename: str):
    # Initialize Qdrant client
    client = qdrant_client.QdrantClient(url="http://65.0.229.53:6333", port=6333)

    COLLECTION_NAME = "alfalah_investment"
    embed_model_name = "text-embedding-3-small"

    # Set the embedding model
    embed_model = OpenAIEmbedding(model_name=embed_model_name)
    Settings.embed_model = embed_model

    # Perform the delete operation
    try:
        response = client.delete(
            collection_name=COLLECTION_NAME,  # Use the variable directly
            points_selector=models.FilterSelector(
                filter=models.Filter(
                    must=[
                        models.FieldCondition(
                            key="filename",
                            match=models.MatchValue(value=filename),
                        )
                    ]
                )
            ),
        )
        print(f"File for the year '{filename}' has been deleted successfully.")
    except Exception as e:
        print(f"Error deleting the file: {e}")

In [9]:
delete_data_from_qdrant("Islamic Fund Manager Report – February 2025.md")

File for the year 'Islamic Fund Manager Report – February 2025.md' has been deleted successfully.


<h1> Add data to Qdrant</h1>


In [6]:
from pprint import pprint


def add_data_to_qdrant(path):

    client = qdrant_client.QdrantClient(url="http://65.0.229.53:6333", port=6333)

    COLLECTION_NAME = "alfalah_investment"
    embed_model_name = "text-embedding-3-small"

    embed_model = OpenAIEmbedding(model_name=embed_model_name)
    Settings.embed_model = embed_model

    vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)
    nodes_data = creat_node_data_from_input_dir(path)
    nodes = create_nodes_from_nodes_data(nodes_data)

    print(nodes)

    logging.info("no collection found")
    storage_context = StorageContext.from_defaults(
        vector_store=vector_store,
    )
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    return index

In [10]:
add_data_to_qdrant("latest_modified_fmr_data/single_modified_file")

[TextNode(id_='a494ef21-76a3-4e5d-8d4e-4cd424adc592', embedding=None, metadata={'year': '2025', 'month': 'February', 'filename': 'Conventional Fund Manager Report – February 2025.md', 'file_id': '44758056-1c40-4d56-b347-e5671c860d5c'}, excluded_embed_metadata_keys=['file_id'], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='44758056-1c40-4d56-b347-e5671c860d5c', node_type=None, metadata={'filename': 'Conventional Fund Manager Report – February 2025.md'}, hash=None)}, metadata_template='{key}: {value}', metadata_separator='\n', text="Fund Managers' Report\n**Date:** February, 2025\n**Company:** Alfalah Investments  \n---", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), TextNode(id_='78509d4c-8a38-44c4-84f8-ced1c0396af7', embedding=None, metadata={'year': '2025', 'month': 'February', 'filename': 'Conventional Fund Manager Report – February 2025.md'

<h1> Prcesssing md Files</h1>


In [15]:
import os


def replace_in_files(folder_path: str):
    """
    Reads all files in the given folder, replaces specific headers with their updated versions,
    and saves the changes back to the files.

    Args:
        folder_path (str): The path to the folder containing the files.
    """
    # Mapping of original headers to their replacements
    replacements = {
        "\nSindh Workers": "\n## Sindh Workers",
        "\n(Holdings as % of Total Assets)": "\n### (Holdings as % of Total Assets)",
        "\nHoldings as % of Total Assets": "\n### (Holdings as % of Total Assets)",
        "\nFund Statistics:": "\n### Fund Statistics:",
        "\nFund Statistic": "\n### Fund Statistic:",
        "\nFund Statistics": "\n### Fund Statistics:",
        "\nFund Stataistics": "\n### Fund Stataistics:",
        "\nfund statistics": "\n### fund statistics:",
        "\nTop Ten Holdings (as a % of total assets)": "\n### Top Ten Holdings (as a % of total assets)",
        "\nSector Allocation (as a % of total assets)": "\n### Sector Allocation (as a % of total assets)",
        "\nAsset Allocation (as % of Total Assets)": "\n### Asset Allocation (as % of Total Assets)",
        "\nRisk Profile:": "\n## Risk Profile:",
        "\nRisk Profile": "\n## Risk Profile:",
        "\nFund Performance": "\n### Fund Performance",
        "\nFund Performanace": "\n### Fund Performanace",
        "\nFund Performanace:": "\n### Fund Performanace:",
        "\nFund Perfomance": "\n### Fund Perfomance",
        "\nfund performance": "\n### Fund Performance",
        "\nPerformance": "\n### Performance",
        "\nAsset Allocation": "\n### Asset Allocation",
        "\n### RISK PROFILE OF ISLAMIC COLLECTIVE INVESTMENT SCHEMES/PLANS": "\n# RISK PROFILE OF ISLAMIC COLLECTIVE INVESTMENT SCHEMES/PLANS",
        "\n### RISK PROFILE OF CONVENTIONAL COLLECTIVE INVESTMENT SCHEMES/PLANS": "\n# RISK PROFILE OF CONVENTIONAL COLLECTIVE INVESTMENT SCHEMES/PLANS",
    }

    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                # Read the file
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()

                # Apply all replacements
                for original, replacement in replacements.items():
                    content = content.replace(original, replacement)

                # Write back the updated content
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(content)

                print(f"Processed file: {file_path}")
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")


# Example usage
folder_path = "latest_modified_fmr_data/single_modified_file"
replace_in_files(folder_path)

Processed file: latest_modified_fmr_data/single_modified_file/Islamic Fund Manager Report – December 2024_1.md
Processed file: latest_modified_fmr_data/single_modified_file/Conventional Fund Manager Report – December 2024_0.md


<h1> Create Filters</h1>


In [3]:
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition,
)

In [1]:
def create_filters_for_all_data(month, year):
    filters_list = []

    # If a month is found, add a month filter
    if month:
        filters_list.append(
            MetadataFilter(key="month", operator=FilterOperator.EQ, value=month)
        )

    # If a year is found, add a year filter
    if year:
        filters_list.append(
            MetadataFilter(key="year", operator=FilterOperator.EQ, value=year)
        )

    # Return filters if any are found, otherwise None
    print(filters_list)
    if filters_list:
        return MetadataFilters(filters=filters_list, condition=FilterCondition.AND)
    return MetadataFilters(
        filters=[
            # MetadataFilter(key="year", operator=FilterOperator.NIN, value=list(range(2014, 2024))),
            MetadataFilter(
                key="year", operator=FilterOperator.IS_EMPTY, value=None
            )  # Avoid including `value`
        ],
        condition=FilterCondition.OR,
    )

In [4]:
create_filters_for_all_data("nov", "2024")

[MetadataFilter(key='month', value='nov', operator=<FilterOperator.EQ: '=='>), MetadataFilter(key='year', value='2024', operator=<FilterOperator.EQ: '=='>)]


MetadataFilters(filters=[MetadataFilter(key='month', value='nov', operator=<FilterOperator.EQ: '=='>), MetadataFilter(key='year', value='2024', operator=<FilterOperator.EQ: '=='>)], condition=<FilterCondition.AND: 'and'>)

In [10]:
def create_filters_for_specific_files(file1, file2):
    filters_list = []

    # Add a filter for the first file
    if file1:
        filters_list.append(
            MetadataFilter(key="filename", operator=FilterOperator.EQ, value=file1)
        )

    # Add a filter for the second file
    if file2:
        filters_list.append(
            MetadataFilter(key="filename", operator=FilterOperator.EQ, value=file2)
        )

    # Combine the filters with an OR condition to get results from both files
    return MetadataFilters(filters=filters_list, condition=FilterCondition.OR)

In [ ]:
def make_filter_for_specific_file(query):
    # Extract month and year from the query
    file1 = "Conventional_all_Fund_Data.md"
    file2 = "Islamic_all_Fund_Data.md"
    # Create and return filters based on extracted month and year
    return create_filters_for_specific_files(file1, file2)

In [5]:
from src.gen_pipeline import GenPipeline
from src.utils import make_filter

gen_pipeline = GenPipeline()
index = gen_pipeline._get_qdrant_index()

/home/osama/Desktop/intermediate-pilot/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/osama/Desktop/intermediate-pilot/venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
query = "tell me fund performance for all funds "
retriever = index.as_retriever(
    similarity_top_k=20, filters=make_filter_for_specific_file(query)
)
retrieve_nodes = retriever.retrieve(query)
retrieve_nodes

[NodeWithScore(node=TextNode(id_='fd89fc6b-8389-4664-9583-cb9bb8e2640b', embedding=None, metadata={'filename': 'Conventional_all_Fund_Data.md', 'file_id': 'cdf49f95-a32e-42ce-9622-eab70e23b53f'}, excluded_embed_metadata_keys=['file_id'], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cdf49f95-a32e-42ce-9622-eab70e23b53f', node_type=None, metadata={'filename': 'Conventional_all_Fund_Data.md'}, hash=None)}, text='Alfalah Financial Sector Income Fund-Plan 1\nFund Stability Rating: "A+(f)" by PACRA 2-July-2024  \nAMC Rating: "AM1" by VIS 29-Dec-23  \nAMC Rating: "AM1" by PACRA 30-August-24  \nInvestment Objective  \nThe objective of the Alfalah Financial Sector Income Fund is to provide income enhancement and preservation of capital by offering different plans which will invest in prime quality Financial Sector TFCs/Sukuks, Bank deposits, spread transactions and short-term money market instruments.  \nBasic Information  \n| Category  